In [1]:
import pandas as pd

In [2]:
import re

In [3]:
import math

In [4]:
import numpy as np

In [5]:
from nltk.corpus import reuters

### 计算语言模型

In [6]:
def lg_model(corpus): 
    word_rate = {}
    word_count = {}
    for i in corpus:
        tmp = ['<s>'] + i +['<s>']
        for  j,k in enumerate(tmp[:-1]):
            if(k not in word_count.keys()):
                word_count[k] = {tmp[j+1]:1}
            else:
                word_dict = word_count[k]
                if(tmp[j+1] in word_dict.keys()):
                    word_count[k][tmp[j+1]] += 1
                else:
                    word_count[k][tmp[j+1]] = 1
    for i in word_count.keys():
        tmp = word_count[i]
        cnt = sum(tmp.values())
        for k in tmp.keys():
            word_rate[" ".join([i,k])] = 1.00*tmp[k]/cnt
    return word_rate       

### 生成编辑距离为1的所有组合

In [7]:
def edite_distance_1(dictionary,word):
    adds = [chr(i) for i in range(97,123)]
    word_all = set()
    for i in range(len(word)):
        #生成删除一个的字符的字符串
        word_all.add(word[0:i] + word[i+1:])
        #生成加一分字母的字符串
        for j in adds:
            word_all.add(word[0:i] + j + word[i:])
        #生成修改一个字母的字符串
        for j in adds:
            word_all.add(word[0:i] + j + word[i+1:])
    return word_all&dictionary   

### 生成拼写错误的概率

In [8]:
def spell_error_pro(spell_error):
    error_count = {}
    error_dict = {}
    for i in spell_error:
        key = i.split(':')[0].strip()
        tmp_dict = {}
        for j in i.split(':')[1].split(','):
            if('*' in j):
                tmp_dict[j.split('*')[0].strip()] = int(j.split('*')[1].strip())
            else:
                tmp_dict[j.strip()] = 1
        error_count[key] = tmp_dict
    for j in error_count.keys():
        cnt = sum(error_count[j].values())
        tmp_dict = {}
        for k in error_count[j].keys():
            tmp_dict[k] = 1.00*error_count[j][k]/cnt
        error_dict[j] = tmp_dict
    return error_dict

### 拼写纠错

In [20]:
def modify_spell_error(word_rate,error_rate,sentence,dictionary):
    res = []
    for i,j in enumerate(sentence[1:-1]):
        if(j not in dictionary):
            correct_words = edite_distance_1(word=j,dictionary=dictionary)
            if(len(correct_words)<=0):
                continue
            rate = float("-inf")
            correct_word = ''
            for k in correct_words:
                if(k in error_rate.keys()):
                    if(j in error_rate[k].keys()):
                        e_r = error_rate[k][j]
                    else:
                        e_r = 0.00001
                else:
                    e_r = 0.00001
                if(' '.join([sentence[i-1],k]) in word_rate.keys()):
                    e_p_r = word_rate[' '.join([sentence[i-1],k])]
                else:
                    e_p_r = 0.00001
                if(' '.join([k,sentence[i+1]]) in word_rate.keys()):
                    e_b_r = word_rate[' '.join([k,sentence[i+1]])]
                else:
                    e_b_r = 0.00001
                    
                rate_tmp = math.log10(e_b_r) + math.log10(e_p_r) + math.log10(e_r)
                if(rate_tmp> rate):
                    rate = rate_tmp
                    correct_word = k
            res.append((j,correct_word))
    return res

### 启动入口

In [23]:
def main():
    #词典库
    vocab = set([line.rstrip() for line in open('./vocab.txt')])
    #读取拼写错误的文本
    spell_error = set([line.rstrip() for line in open('./spell-errors.txt')])
    #读取需要进行拼写纠错的文本
    input_txt = [i.split('\t') for i in [line for line in open('./testdata.txt')]]
    #读取语料库
    category = reuters.categories()
    corpus = reuters.sents(categories=category)
    word_rate = lg_model(corpus=corpus)
    print(word_rate)
    error_rate = spell_error_pro(spell_error=spell_error)
    for i in input_txt:
        print(i[0],i[1])
        sentence = ['<s>'] + [j.strip().replace('.','').replace(',','').replace("'s",'') for j in i[2].split(' ')] + ['<s>']
        print(modify_spell_error(word_rate=word_rate,error_rate=error_rate,sentence=sentence,dictionary = vocab))
        

In [24]:
main()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




[('growti', 'growth')]
396 1
[('Grman', 'German')]
397 1
[('ibpact', 'impact')]
398 1
[]
399 1
[('obstaclet', 'obstacle')]
400 1
[]
401 1
[]
402 1
[]
403 1
[]
404 1
[]
405 1
[('affecte', 'effecte')]
406 1
[('countris', 'countries')]
407 1
[('aout', 'about')]
408 1
[]
409 1
[('therel', 'there')]
410 1
[]
411 1
[]
412 1
[]
413 1
[('conmmercial', 'commercial')]
414 1
[('trhee', 'tree')]
415 1
[]
416 1
[('Februoary', 'February')]
417 1
[('nted', 'need')]
418 1
[('numbir', 'number')]
419 1
[]
420 1
[]
421 1
[('availabale', 'available')]
422 1
[]
423 1
[('benfit', 'benefit')]
424 1
[('ressult', 'result')]
425 1
[]
426 1
[('beforeb', 'before')]
427 1
[]
428 1
[('proudcer', 'prouder')]
429 1
[('shaer', 'sheer'), ('1539', '139')]
430 1
[('ootstanding', 'outstanding')]
431 1
[('pureec', 'purees')]
432 1
[('sarplus', 'surplus')]
433 1
[('Aprl', 'April')]
434 1
[('prevnted', 'prevented')]
435 1
[('duspite', 'despite')]
436 1
[]
437 1
[('calledi', 'called'), ("workers'", 'workers')]
438 1
[]
439 1

[('specuration', 'speculation'), ('weesk', 'week')]
937 3
[('estaae', 'estate'), ('Trumep', 'Trump'), ('stocsk', 'stock')]
938 3
[('indicwted', 'indicated'), ('posintion', 'position')]
939 2
[('circulaed', 'circulated'), ('Conisnton', 'Coniston')]
940 3
[('juped', 'jumped'), ('shaes', 'shams')]
941 3
[('Unites', 'United'), ('cnsider', 'consider')]
942 3
[('tink', 'think'), ('proposel', 'proposes'), ('reilistic', 'realistic')]
943 2
[('manogement', 'management'), ('Steans', 'Stans')]
944 3
[('Anlysts', 'Analysts')]
945 3
[('adted', 'added'), ('servilce', 'servile')]
946 2
[]
947 3
[('stootd', 'stood'), ('boginning', 'beginning')]
948 3
[('Allegits', 'Allegis')]
949 3
[('valies', 'valves')]
950 3
[('todayl', 'today'), ('yubing', 'tubing'), ('belive', 'believe')]
951 4
[('attractsed', 'attracted'), ('sotck', 'sock'), ('actsed', 'acted'), ('anoteer', 'another')]
952 3
[('firset', 'first'), ('investhor', 'investor'), ('staae', 'stave')]
953 2
[('uninon', 'unison')]
954 2
[('attituode', 'att

In [100]:
category = reuters.categories()
corpus = reuters.sents(categories=category)
word_rate = lg_model(corpus=corpus)
print(word_rate)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [101]:
word_rate

{'<s> ASIAN': 7.31047591198187e-05,
 '<s> They': 0.008151180641859785,
 '<s> But': 0.019263104028072228,
 '<s> The': 0.16154324146501936,
 '<s> Unofficial': 1.8276189779954676e-05,
 '<s> "': 0.06559324512025733,
 '<s> In': 0.02522114189633745,
 '<s> Threat': 3.655237955990935e-05,
 '<s> Taiwan': 0.0006944952116382777,
 '<s> Retaliation': 5.482856933986402e-05,
 '<s> A': 0.013963008991885371,
 '<s> Last': 0.0036917903355508444,
 '<s> Much': 0.0001462095182396374,
 '<s> He': 0.028986036991008116,
 '<s> Meanwhile': 0.0007493237809781417,
 '<s> Japan': 0.0020286570655749687,
 '<s> Deputy': 0.0001462095182396374,
 '<s> CHINA': 0.0009138094889977337,
 '<s> It': 0.03231230353095987,
 '<s> JAPAN': 0.002997295123912567,
 '<s> MITI': 0.0002193142773594561,
 '<s> Nuclear': 1.8276189779954676e-05,
 '<s> THAI': 0.00034724760581913885,
 '<s> Thailand': 0.00023759046713941077,
 '<s> Export': 0.0002193142773594561,
 '<s> Products': 7.31047591198187e-05,
 '<s> INDONESIA': 0.00038379998537904815,
 '<s> 